In [1]:
import requests
import pandas as pd
import sqlalchemy as sa
import pyodbc

## Forecast Weather Data

In [3]:
locations = [
    {'Province': 'Ontario', 'City': 'North York', 'PostalCode': 'M3J1P3', 'Latitude': '43.773784', 'Longitude': '-79.500362'},
    {'Province': 'Manitoba', 'City': 'Winnipeg', 'PostalCode': 'R0G2W0', 'Latitude': '49.798866', 'Longitude': '-97.064934'},
    {'Province': 'Quebec', 'City': 'Saguenay-Lac-Saint-Jean', 'PostalCode': 'G0V0A1', 'Latitude': '48.211288', 'Longitude': '-70.069969'},
    {'Province': 'Saskatchewan', 'City': 'Regina', 'PostalCode': 'S0A0A0', 'Latitude': '50.615061', 'Longitude': '-103.412753'},
    {'Province': 'Ontario', 'City': 'Thunder Bay', 'PostalCode': 'P0M2J0', 'Latitude': '48.68475', 'Longitude': '-85.641417'},
    {'Province': 'Saskatchewan', 'City': 'Saskatoon', 'PostalCode': 'S0G0A6', 'Latitude': '51.627971', 'Longitude': '-106.439486'},
    {'Province': 'Ontario', 'City': 'Sudbury', 'PostalCode': 'P0H0B9', 'Latitude': '46.442016', 'Longitude': '-80.311475'},
    {'Province': 'Alberta', 'City': 'Edmonton', 'PostalCode': 'T0A0A7', 'Latitude': '53.825792', 'Longitude': '-113.326611'},
    {'Province': 'Nunavut', 'City': 'Baffin', 'PostalCode': 'X0A0A0', 'Latitude': '73.005278', 'Longitude': '-85.033056'},
    {'Province': 'Alberta', 'City': 'Edmonton', 'PostalCode': 'T0A0A9', 'Latitude': '53.83726', 'Longitude': '-113.412126'},
    {'Province': 'Quebec', 'City': 'Gaspésie-Îles-de-la-Madeleine', 'PostalCode': 'G0C2Y0', 'Latitude': '48.175265', 'Longitude': '-65.249235'},
    {'Province': 'Nunavut', 'City': 'Keewatin', 'PostalCode': 'X0C0H0', 'Latitude': '66.556724', 'Longitude': '-86.21719'},
    {'Province': 'Northwest Territories', 'City': 'Region 6', 'PostalCode': 'X1A1Y1', 'Latitude': '62.448206', 'Longitude': '-114.373354'}
]

current_key = 'E7O8dbh7EMSzAaOJcal3lj9dJKPkCB7H' 
headers = {"accept": "application/json", 'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"} 

forecast_weather_data = []

for location in locations:
    lat = location['Latitude']
    lon = location['Longitude']
    pro = location['Province']
    city = location['City']
    code = location['PostalCode']
    forecast_url = f'https://api.tomorrow.io/v4/weather/forecast?location={lat},{lon}&timesteps=1d&apikey={current_key}'
    
    response = requests.get(forecast_url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        for timeline in data['timelines']['daily']:
            weather_info = {
                'Latitude': lat,
                'Longitude': lon,
                'Province': pro,
                'City': city,
                'Postal Code': code,
                'Time(UTC)': timeline['time'],
                'Avg_Temperature': timeline['values']['temperatureApparentAvg'],
                'Max_Temperature': timeline['values']['temperatureApparentMax'],
                'Min_Temperature': timeline['values']['temperatureApparentMin'],
                'Humidity': timeline['values']['humidityAvg']
            }
            forecast_weather_data.append(weather_info)
    else:
        print(f"Failed to fetch data for location ({lat}, {lon})")
        print(response.text)

weather_df = pd.DataFrame(forecast_weather_data)

print(weather_df)

Failed to fetch data for location (53.83726, -113.412126)
{"code":429001,"type":"Too Many Calls","message":"The request limit for this resource has been reached for the current rate limit window. Wait and retry the operation, or examine your API request volume."}
     Latitude    Longitude               Province        City Postal Code  \
0   43.773784   -79.500362                Ontario  North York      M3J1P3   
1   43.773784   -79.500362                Ontario  North York      M3J1P3   
2   43.773784   -79.500362                Ontario  North York      M3J1P3   
3   43.773784   -79.500362                Ontario  North York      M3J1P3   
4   43.773784   -79.500362                Ontario  North York      M3J1P3   
..        ...          ...                    ...         ...         ...   
67  62.448206  -114.373354  Northwest Territories    Region 6      X1A1Y1   
68  62.448206  -114.373354  Northwest Territories    Region 6      X1A1Y1   
69  62.448206  -114.373354  Northwest Terri

## Feature Engineering

In [4]:
weather_df['Temperature_Range'] = weather_df['Max_Temperature'] - weather_df['Min_Temperature']

In [7]:
weather_df['Time(UTC)'] = pd.to_datetime(weather_df['Time(UTC)'])
weather_df['Day'] = weather_df['Time(UTC)'].dt.day
weather_df['Month'] = weather_df['Time(UTC)'].dt.month
weather_df['Hour'] = weather_df['Time(UTC)'].dt.hour

In [8]:
weather_df

,Latitude,Longitude,Province,City,Postal Code,Time(UTC),Avg_Temperature,Max_Temperature,Min_Temperature,Humidity,Temperature_Range,Day,Month,Hour
0,43.773784,-79.500362,Ontario,North York,M3J1P3,2024-08-02 10:00:00+00:00,25.16,29.16,21.75,75.41,7.41,2,8,10
1,43.773784,-79.500362,Ontario,North York,M3J1P3,2024-08-03 10:00:00+00:00,24.97,32.28,18.50,65.86,13.78,3,8,10
2,43.773784,-79.500362,Ontario,North York,M3J1P3,2024-08-04 10:00:00+00:00,24.60,30.91,18.00,61.61,12.91,4,8,10
3,43.773784,-79.500362,Ontario,North York,M3J1P3,2024-08-05 10:00:00+00:00,22.04,28.03,16.85,69.95,11.18,5,8,10
4,43.773784,-79.500362,Ontario,North York,M3J1P3,2024-08-06 10:00:00+00:00,19.91,24.36,15.66,66.80,8.70,6,8,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,62.448206,-114.373354,Northwest Territories,Region 6,X1A1Y1,2024-08-03 12:00:00+00:00,17.60,20.31,14.31,59.06,6.00,3,8,12
68,62.448206,-114.373354,Northwest Territories,Region 6,X1A1Y1,2024-08-04 12:00:00+00:00,18.09,22.02,14.17,65.55,7.85,4,8,12
69,62.448206,-114.373354,Northwest Territories,Region 6,X1A1Y1,2024-08-05 12:00:00+00:00,17.35,20.65,13.73,55.47,6.92,5,8,12
70,62.448206,-114.373354,Northwest Territories,Region 6,X1A1Y1,2024-08-06 12:00:00+00:00,16.81,19.91,13.54,47.27,6.37,6,8,12


In [9]:
# Database Connection
connection_url = sa.engine.URL.create(
    drivername = "mssql+pyodbc",
    username   = "aimerliu",
    password   = "2024!Schulich",
    host       = "mban2024-ms-sql-server.c1oick8a8ywa.ca-central-1.rds.amazonaws.com",
    port       = "1433",
    database   = "aimerliu_db",
    query = {
        "driver" : "ODBC Driver 18 for SQL Server",
        "TrustServerCertificate" : "yes"
    }
)
my_engine = sa.create_engine(connection_url)

In [10]:
# Ingest the data into your own database in our Microsoft SQL Server
weather_df.to_sql(
    name='Final Forecasting Weather Data',
    con=my_engine,
    schema = 'uploads',
    if_exists='replace',
    index=False,
    dtype= {
        'Latitude': sa.types.DECIMAL(10,3),
        'Longitude': sa.types.DECIMAL(10,3),
        'Province': sa.types.VARCHAR(30),
        'City': sa.types.VARCHAR(40),
        'Postal Code': sa.types.VARCHAR(10),
        'Time(UTC)': sa.types.DATETIME,
        'Avg_Temperature': sa.types.FLOAT,
        'Max_Temperature': sa.types.FLOAT,
        'Min_Temperature': sa.types.FLOAT,
        'Humidity': sa.types.FLOAT,
        'Temperature_Range': sa.types.DECIMAL(10,3),
        'Day': sa.types.Integer,
        'Month': sa.types.Integer,
        'Hour': sa.types.Integer
    },
    method='multi'
)

72